In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data =  pd.read_csv('combo.gzip',compression='gzip',parse_dates=True,index_col='date')
print('done');

done


In [88]:
data.arrival_schedule = data.arrival_schedule.fillna(value=(data.arrival_actual + data.arrival_delay))
data.departure_schedule = data.departure_schedule.fillna(value=(data.departure_actual + data.departure_delay))

In [77]:
#departure&arrival_schedule are now not null and int32

In [94]:
for column in data:
    assert pd.notnull(data[column]).all()
    print(column+' is clean')

airline is clean
airline_code is clean
departure_airport is clean
departure_state is clean
departure_lat is clean
departure_lon is clean
arrival_airport is clean
arrival_state is clean
arrival_lat is clean
arrival_lon is clean
departure_schedule is clean
departure_actual is clean
departure_delay is clean
arrival_schedule is clean
arrival_actual is clean
arrival_delay is clean


In [99]:
data['airline_code'] = data['airline_code'].astype(str)
data['departure_delay'] = data['departure_delay'].astype(float)
data['arrival_delay'] = data['arrival_delay'].astype(float)
data['arrival_schedule'] = data['arrival_schedule'].astype(int)
data['departure_schedule'] = data['departure_schedule'].astype(int)

In [107]:
data = data.reset_index()

In [108]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11401196 entries, 0 to 11401195
Data columns (total 17 columns):
date                  datetime64[ns]
airline               object
airline_code          object
departure_airport     object
departure_state       object
departure_lat         float64
departure_lon         float64
arrival_airport       object
arrival_state         object
arrival_lat           float64
arrival_lon           float64
departure_schedule    int32
departure_actual      int64
departure_delay       float64
arrival_schedule      int32
arrival_actual        int64
arrival_delay         float64
dtypes: datetime64[ns](1), float64(6), int32(2), int64(2), object(6)
memory usage: 1.4+ GB
